In [ ]:
from frontend import App

app = App("trapped", renew=True)

V, F, T = app.mesh.preset("armadillo").decimate(30000).tetrahedralize().normalize()
app.asset.add.tet("armadillo", V, F, T)

scene = app.scene.create("sphere-trap")

scene.add.invisible.sphere([0,0,0],0.7).invert().radius(0.15,3).radius(100,4)
scene.add("armadillo").jitter().rotate(180,"y")

fixed = scene.build().report()
fixed.preview();

In [ ]:
param = app.session.param()
param.set("gravity", 0.0);
param.set("dt", 0.01);

session = app.session.create("armadillo-trapped").init(fixed)
session.start(param).preview();
session.stream();

In [ ]:
# run this cell after sufficnt frames are simulated
session.animate();

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
# export all simulated frames
session.export.animation(f"export/{scene.info.name}/{session.info.name}")